#  Importing numpy,pandas,XGBClassifier and train_test_split.

In [71]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from xgboost.sklearn import XGBClassifier
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/toxicity-competetion/sample_submission.csv
/kaggle/input/toxicity-competetion/feamat.csv
/kaggle/input/toxicity-competetion/features_id_name_mappings.csv
/kaggle/input/toxicity-competetion/train.csv
/kaggle/input/toxicity-competetion/test.csv


# Loading test , train and feamat datasets

In [72]:
train= pd.read_csv("../input/toxicity-competetion/train.csv")
feamat=pd.read_csv("../input/toxicity-competetion/feamat.csv")
test=pd.read_csv("../input/toxicity-competetion/test.csv")
y=train["Expected"]



# Removing ';' between ChemId and AssayId

In [73]:
train[['ChemId','AssayId']]=train.Id.str.split(";",expand=True)
test[['ChemId','AssayId']]=test.x.str.split(";",expand=True)
#feamat.rename(columns={'V1':'ChemId'},inplace=True)

# Replacing infinity values with null and then with mean

In [74]:
feamat.replace([np.inf, -np.inf], np.nan, inplace=True)
feamat['V15'].fillna(value=feamat['V15'].mean(), inplace=True)
feamat.columns

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       ...
       'V1066', 'V1067', 'V1068', 'V1069', 'V1070', 'V1071', 'V1072', 'V1073',
       'V1074', 'V1075'],
      dtype='object', length=1075)

# Dropping all the colmns after V29

In [75]:
feamat.drop(feamat.iloc[:, 29:1075], inplace= True, axis=1)
feamat.columns

Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
       'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'V29'],
      dtype='object')

# Merging Featmat with test and train datasets

In [76]:
feamat.rename(columns={'V1':'ChemId'}, inplace=True)
train=train.merge(feamat, on="ChemId", how= "left")
test=test.merge(feamat, on="ChemId", how="left")

# Train and Test datasets after merging

In [77]:
train

,Id,Expected,ChemId,AssayId,V2,V3,V4,V5,V6,V7,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29
0,2971-36-0;1644,2,2971-36-0,1644,76302,315.982463,4.592,40.46,0.0,0.0,...,0,2,43.687788,26.519496,59.322530,41.038992,52.085432,290,5.321928,1
1,693-54-9;2451,2,693-54-9,2451,12741,156.151415,3.852,17.07,0.0,0.0,...,1,0,11.774469,10.332092,10.664184,8.332092,7.332092,139,4.321928,1
2,7173-51-5;1384,2,7173-51-5,1384,23558,361.347528,9.912,0.00,0.0,0.0,...,0,0,32.072930,22.664728,24.332364,22.332364,21.332364,1050326996,5.459432,1
3,138261-41-3;16,2,138261-41-3,16,86418,255.052302,2.294,83.66,0.0,0.0,...,4,1,28.061789,22.748130,32.352497,26.617141,23.522689,273,5.169925,1
4,7681-82-5;1856,2,7681-82-5,1856,5238,149.894242,1.050,0.00,0.0,0.0,...,0,0,115.302448,0.000000,0.000000,0.000000,0.000000,0,0.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77408,3337-71-1;33,2,3337-71-1,33,18752,230.036128,0.111,106.87,0.0,0.0,...,6,2,24.945590,25.224679,30.934870,31.607923,25.928544,198,4.906891,1
77409,42509-80-8;1632,1,42509-80-8,1632,39223,313.041677,3.740,100.30,0.0,0.0,...,4,0,40.894465,35.329639,47.935705,45.904321,47.194958,250,5.169925,1
77410,149-30-4;1373,1,149-30-4,1373,697993,166.986341,2.115,69.42,0.0,0.0,...,1,1,22.615486,16.341719,29.026598,18.011505,11.175538,87,4.459432,1
77411,123-05-7;2,2,123-05-7,2,31241,128.120115,3.156,17.07,0.0,0.0,...,1,0,9.774469,8.332092,8.332092,8.664184,6.664184,74,4.000000,1


In [78]:
test

,x,ChemId,AssayId,V2,V3,V4,V5,V6,V7,V8,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29
0,88-60-8;1682,88-60-8,1682,6937,164.120115,3.659,20.23,0.0,0.0,0.000000,...,0,1,12.774469,12.332092,18.664184,16.996276,14.660460,106,4.584963,1
1,122931-48-0;1656,122931-48-0,1656,91779,431.056940,1.258,183.30,0.0,0.0,0.000000,...,10,2,47.476691,49.504506,62.259656,71.196703,77.917913,590,5.857981,1
2,NOCAS_47311;36,NOCAS_47311,36,53257735,695.250845,6.365,95.92,0.0,0.0,0.174792,...,7,1,68.968100,68.337655,99.856240,110.392516,108.648356,1715,6.754888,1
3,55589-62-3;1850,55589-62-3,1850,11074431,200.949810,-1.340,68.82,0.0,0.0,0.000000,...,5,0,30.182453,20.613020,24.039282,19.054704,13.150049,-1474836500,4.321928,1
4,79902-63-9;30,79902-63-9,30,54454,418.271924,4.775,72.83,0.0,0.0,0.000000,...,5,1,33.872345,34.324644,51.869858,55.313472,51.866134,632,6.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11134,141517-21-7;38,141517-21-7,38,213016,408.129692,5.760,69.48,0.0,0.0,0.000000,...,2,0,37.323969,34.845102,50.683508,52.417371,53.022116,668,5.906891,1
11135,81-90-3;34,81-90-3,34,66494,320.104859,4.458,77.76,0.0,0.0,0.000000,...,2,3,27.097876,27.328368,38.767021,40.656736,40.656736,390,5.700440,1
11136,74223-64-6;1640,74223-64-6,1640,52999,381.074304,1.320,157.85,0.0,0.0,0.000000,...,10,2,38.574469,39.384229,51.865859,56.322681,61.384213,542,5.754888,1
11137,62-73-7;28,62-73-7,28,3039,219.945901,1.207,54.57,0.0,0.0,0.000000,...,4,0,35.174190,24.640823,34.331916,22.431727,22.220562,94,4.321928,1


# Dropping ChemId, Id, V2 from train and test datasets

In [79]:
train.drop(['ChemId','Id', 'V2','Expected'], axis=1, inplace= True)
test.drop(['ChemId', 'x', 'V2'], axis=1 , inplace=True)

# Considerig columns into feature array and getting final test and training datasets

In [80]:
features=[ 'AssayId','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24','V25','V26','V27','V28']
mapped_train= train[features]
mapped_test= test[features]

# Loading test datasets to combine with the resultant predicted value

In [81]:
test1=pd.read_csv("../input/toxicity-competetion/test.csv")

# Converting AssaId Object datatype to Int datatype

In [82]:
mapped_train['AssayId'] = mapped_train['AssayId'].astype(str).astype(int)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [83]:
mapped_train.dtypes

AssayId      int64
V4         float64
V5         float64
V6         float64
V7         float64
V8         float64
V9         float64
V10        float64
V11        float64
V12        float64
V13        float64
V14        float64
V15        float64
V16        float64
V17          int64
V18          int64
V19          int64
V20          int64
V21          int64
V22        float64
V23        float64
V24        float64
V25        float64
V26        float64
V27          int64
V28        float64
dtype: object

In [84]:
mapped_test['AssayId'] = mapped_test['AssayId'].astype(str).astype(int)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [85]:
mapped_test.dtypes

AssayId      int64
V4         float64
V5         float64
V6         float64
V7         float64
V8         float64
V9         float64
V10        float64
V11        float64
V12        float64
V13        float64
V14        float64
V15        float64
V16        float64
V17          int64
V18          int64
V19          int64
V20          int64
V21          int64
V22        float64
V23        float64
V24        float64
V25        float64
V26        float64
V27          int64
V28        float64
dtype: object

# Calculating f1_score and accuracy_score with trai_test_split

In [86]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,accuracy_score

train_X, val_X, train_y, val_y = train_test_split(mapped_train, y, random_state=42)


clf=XGBClassifier(n_estimators=250, max_dept=12, random_state=1,Learning_rate=0.25,booster='gbtree')
#clf = RandomForestClassifier()
#clf = DecisionTreeClassifier(criterion='entropy')
clf.fit(train_X,train_y)
y_pred=clf.predict(val_X)

macro = f1_score(val_y, y_pred, average='macro')
print('Macro Score:',macro)
acc=accuracy_score(val_y,y_pred)
acc

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:20:56] WARNING: ../src/learner.cc:541: 
Parameters: { Learning_rate, max_dept } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:20:56] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Macro Score: 0.7762493402284444


0.9041025111088147

# Taking XGBClassifier as main model and creating the final submission file

In [87]:
MainModel = XGBClassifier(n_estimators=250, max_dept=12, random_state=1,Learning_rate=0.25,booster='gbtree')
MainModel.fit(mapped_train,y)
Yprediction = MainModel.predict(mapped_test)
output_file= pd.DataFrame({'Id':test1.x, 'Predicted': Yprediction})
print(output_file)
output_file.to_csv('Submission.csv', index=False)
print("Output file ceated")

[12:21:07] WARNING: ../src/learner.cc:541: 
Parameters: { Learning_rate, max_dept } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[12:21:07] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


                     Id  Predicted
0          88-60-8;1682          2
1      122931-48-0;1656          2
2        NOCAS_47311;36          2
3       55589-62-3;1850          2
4         79902-63-9;30          2
...                 ...        ...
11134    141517-21-7;38          2
11135        81-90-3;34          2
11136   74223-64-6;1640          2
11137        62-73-7;28          2
11138    2634-33-5;1855          1

[11139 rows x 2 columns]
Output file ceated
